In [1]:
import pandas as pd
import numpy as np
import random
import math
import scipy as sp
import scipy.stats as stats
import scipy.signal as correlate
from scipy.interpolate import interp1d
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from matplotlib import cm
from sklearn.linear_model import LinearRegression

# A & R Clear Lake Counts

This is an experimental example of how pollen data can be analyzed and plotted in python. In this example, two counts of the same slides from different researchers are compared to gain an understanding of potential mistakes, tendencies, and sample trends. This is only intended as a basic template which may be informative for other researchers interested in digitzing aspects of their pollen data analysis. 

Please feel free to contact the author with any questions, or for a customized python template to suit your paleoecological research needs: 

Fall 2019 | Violet AB Walsh | violet@buxwal.com

In [2]:
#Import Data
allCLpollen = pd.read_csv('./CLPollenTotal/CombinedCL.csv',skiprows=1,delimiter=';')
A_counts = pd.read_csv('./CLPollenTotal/m-counts-raw.csv')
rcounts = pd.read_csv('./CLPollenTotal/r-counts-raw.csv')

#Drop replace NaNs with 0 and show data
allCLpollen = allCLpollen.fillna(0)

In [3]:
#A_counts_clean.add() = A_counts_clean['Pinus'].add() + A_counts_clean['Indeterminate Conifer']

In [4]:
#####REDISTRIBUTES COUNTS TO A CATEGORIES#####

#add pine and white pine together into one pine category
allCLpollen['Pinus'] = allCLpollen['Pinus (white)'] + allCLpollen['Pinus']

#add juniperus/cupressus and TCT together into one TCT category
allCLpollen['TCT'] = allCLpollen['Juniperus/Cupressus'] + allCLpollen['TCT']

#add quercus and Q. chrysolepis/Q. vaccinifolia and cercocarpus into one quercus category
allCLpollen['Quercus'] = allCLpollen['Quercus'] + allCLpollen['Q. chrysolepis/Q. vaccinifolia'] + allCLpollen['Cercocarpus']

#Drop Roger's extra categories, useing A's only
a = allCLpollen.drop('Pinus (white)',axis=1)
b = a.drop('Juniperus/Cupressus',axis=1)
c = b.drop('Cercocarpus',axis = 1)
simpleCL = c.drop('Q. chrysolepis/Q. vaccinifolia',axis=1)

In [5]:
#replace NaNs with 0s
A_counts = A_counts.fillna(0)
rcounts = rcounts.fillna(0)

#remove columns of notes such that dfs only contain data and have the same indices
a = A_counts.drop('Unnamed: 0', axis=1)
b = a.drop('Unnamed: 1', axis=1)
A_counts = b.drop('Unnamed: 2', axis=1)

##use genus labels from row 1 for column headers
A_counts = A_counts.rename(columns=A_counts.iloc[0])
A_counts = A_counts.drop(0, axis=0)

rcounts = rcounts.rename(columns=rcounts.iloc[0])
rcounts = rcounts.drop(0, axis=0)

##Standardize column names for depth and pollen sums
A_counts = A_counts.rename(columns={"Roger's Depth(Original Field Depths)": "Depth"})
A_counts_clean = A_counts.rename(columns={"Sum of all pollen": "Total"})

rcounts_clean = rcounts.rename(columns={"Total Pollen and Spores": "Total"})

##make sure type(data) == str, this causes problems graphing later
A_counts_clean["Depth"] = A_counts_clean["Depth"].apply(float)
A_counts_clean["Total"] = A_counts_clean["Total"].apply(float)

rcounts_clean["Depth"] = rcounts_clean["Depth"].apply(float)
rcounts_clean["Total"] = rcounts_clean["Total"].apply(float)

##standardize categories##
rcounts_clean['Quercus'] = rcounts_clean['Quercus'].apply(float)
rcounts_clean['Cercocarpus'] = rcounts_clean['Cercocarpus'].apply(float)
rcounts_clean['Q. chrysolepis/Q. vaccinifolia'] = rcounts_clean['Q. chrysolepis/Q. vaccinifolia'].apply(float)
rcounts_clean['Quercus'] = rcounts_clean['Quercus'] + rcounts_clean['Cercocarpus'] + rcounts_clean['Q. chrysolepis/Q. vaccinifolia']

rcounts_clean = rcounts_clean.drop('Cercocarpus', axis = 1)
rcounts_clean = rcounts_clean.drop('Q. chrysolepis/Q. vaccinifolia', axis = 1)

In [6]:
#####DROPS EMPTY COLUMNS###### 

#return the new columns, which should match the categories A counted
new_cols = simpleCL.columns
new_cols = new_cols.drop('Depth')
new_cols = new_cols.drop('LcC or Uch')

#sums each column to find absent types
type_sum = simpleCL.sum(axis=0)

print('The following empty columns will be dropped from the total count data:')
for i in new_cols:
    type_sum = type_sum[new_cols]
    if type_sum[i] == 0:
        simpleCL = simpleCL.drop(i,axis=1)
        print(i)

The following empty columns will be dropped from the total count data:
Fremontia
Adenostoma
Sambucus
Shepherdia
Elaeagnus
Sanguisorba
Ribes
Purshia
Linaceae
Onagraceae
Vitis
Rumex 
Lonicera
Equisetum
Ruppia
Menyanthes
Athyrium
Polystichum
Unknown A
Unknown B
Unknown C
Unknown D
Unknown E
Unknown F


In [7]:
for i in new_cols:
    type_sum = type_sum[new_cols]
    type_sum = type_sum.sort_values(ascending=False)
print(type_sum.to_string())

APFAC                      87570
Total Pollen and Spores    21300
Zea                        17060
Lycopodium                 10713
Pinus                       7434
Quercus                     4394
TCT                         2786
Nuphar hydropotes            684
Alnus                        572
Other unknowns               496
Indeterminate                464
Artemisia                    460
Compositae HS                460
Cyperaceae                   376
Isoetes                      374
Coprophilous fungi           353
Abies                        301
Fraxinus                     291
Poaceae                      246
Salix                        176
Colonial fungi               163
Typha latifolia              108
Pseudotsuga                  105
Tsuga mertensiana             95
Pteridium                     84
Picea                         83
Tsuga heterophylla            75
Amaranthaceae                 75
Garrya                        54
Umbelliferae                  54
Corylus   

In [8]:
#assigns present pollen types to the name column list 'present_types'
present_types = simpleCL.columns
print(present_types)

Index(['Depth', 'LcC or Uch', 'Abies', 'Picea', 'Pinus', 'Ephedra',
       'Pseudotsuga', 'TCT', 'Tsuga mertensiana', 'Tsuga heterophylla', 'Acer',
       'Acer negundo', 'Quercus', 'Aesculus', 'Lithocarpus/Castanopsis',
       'Fraxinus', 'Juglans', 'Garrya', 'Platanus', 'Rosaceae', 'Rhamnaceae',
       'Ceanothus', 'Cephalanthus', 'Salix', 'Myrica', 'Corylus', 'Alnus',
       'Artemisia', 'Ambrosia', 'Liguliflorae', 'Compositae HS',
       'Compositae LS', 'Ericaceae', 'Cruciferae', 'Arceuthobium',
       'Caryophyllaceae', 'Malvaceae', 'Liliaceae', 'Leguminosae', 'Labiatae',
       'Salvia', 'Eriogonum', 'Polygonum amphibium', 'Polygonum californica',
       'Plantago', 'Polemoniaceae', 'Navarettia', 'Ranunculaceae',
       'Umbelliferae', 'Rubiaceae', 'Thalictrum', 'Rhus', 'Amaranthaceae',
       'Sarcobatus', 'Poaceae', 'Cyperaceae', 'Typha latifolia',
       'Typha/Sparganium', 'Potamogeton', 'Isoetes', 'Myriophyllum',
       'Nuphar hydropotes', 'Nuphar', 'Brasenia', 'Dryopteris

In [9]:
###Test Cell, to demonstrate accuracy of graphed data###
simpleCL["Pinus"]
simpleCL["Total Pollen and Spores"]
simpleCL["Depth"]

simpleCL.head()

,Depth,LcC or Uch,Abies,Picea,Pinus,Ephedra,Pseudotsuga,TCT,Tsuga mertensiana,Tsuga heterophylla,...,Trilete,Monolete,Colonial fungi,Coprophilous fungi,Indeterminate,Other unknowns,Zea,Lycopodium,APFAC,Total Pollen and Spores
0,"13,64",LcC,0,0,54,0,1,8,25,1,...,0,1,3,6,15,24,560,241,2085,422
1,"14,56",LcC,1,0,55,0,1,31,0,2,...,0,0,2,25,26,30,310,223,2085,477
2,"15,51",LcC,4,0,60,0,0,39,3,0,...,1,2,10,19,26,17,258,207,2085,561
3,"16,48",LcC,5,1,69,0,0,42,1,0,...,0,0,3,21,0,0,288,231,2085,486
4,"17,07",LcC,0,1,113,0,1,44,0,0,...,0,0,8,8,17,11,367,136,2085,531


In [10]:
A_counts_clean.head()

,Depth,Indeterminate Conifer,Abies,Picea,Pinus,Pinus (white),Ephedra,Pseudotsuga,TCT,Juniperus/Cupressus,...,Unknown E,Unknown F,Indeterminate,Other unknowns,Zea,Lycopodium,APFAC,Total,0.0,0.0
1,9.43,7,0,0,25,0,0,0,9,0,...,0,0,109,50,0,31,0,293.0,0.0,0.0
2,16.85,5,0,2,28,0,0,3,48,0,...,0,0,74,3,72,181,0,426.0,0.0,0.0
3,17.89,3,0,5,37,0,0,0,59,0,...,0,0,57,7,67,198,0,444.0,0.0,0.0
4,18.91,0,3,7,66,0,0,1,30,0,...,0,0,28,22,285,211,0,401.0,0.0,0.0
5,19.31,0,0,5,84,0,0,2,51,0,...,0,0,39,18,307,302,0,621.0,0.0,0.0


In [11]:
rcounts_clean.head()

,Depth,Abies,Picea,Pinus,Pinus (white),Ephedra,Pseudotsuga,TCT,Juniperus/Cupressus,Tsuga mertensiana,...,Unknown C,Unknown D,Unknown E,Unknown F,Indeterminate,Other unknowns,Zea,Lycopodium,APFAC,Total
1,13.64,0,0,54,0,0,1,0,8,25,...,0,0,0,0,15,24,560,241,2085,422.0
2,14.56,1,0,55,0,0,1,26,5,0,...,0,0,0,0,26,30,310,223,2085,477.0
3,15.51,4,0,60,0,0,0,34,5,3,...,0,0,0,0,26,17,258,207,2085,561.0
4,16.48,5,1,69,0,0,0,39,3,1,...,0,0,0,0,0,0,288,231,2085,486.0
5,17.07,0,1,113,0,0,1,37,7,0,...,0,0,0,0,17,11,367,136,2085,531.0


In [12]:
print(A_counts_clean.columns)

Index([                'Depth', 'Indeterminate Conifer',
                       'Abies',                 'Picea',
                       'Pinus',         'Pinus (white)',
                     'Ephedra',           'Pseudotsuga',
                         'TCT',   'Juniperus/Cupressus',
       ...
                   'Unknown E',             'Unknown F',
               'Indeterminate',        'Other unknowns',
                         'Zea',            'Lycopodium',
                       'APFAC',                 'Total',
                           0.0,                     0.0],
      dtype='object', length=109)


In [13]:
new_AcolsDF.head()

NameError: name 'new_AcolsDF' is not defined

In [ ]:
####RETURN actual replicate levels####
levels_m = A_counts_clean['Depth']
levels_r = rcounts_clean['Depth']

simpleCL["Depth"]
print('CL depth 29.12m is the only replicate level')

####USEFUL ONLY FOR MULTIPLE REPLICATE LEVELS###
#replicate_levels = []
#if i in replicate_levels:
    #difbylevelpicea = A_counts_clean['Picea'][i] - rcounts_clean['Picea'][i]
    #difbylevelpicea/len(replicate_levels)

In [ ]:
##chose which species to plot by listing them in geni = []
geni = ["Pinus","TCT"]

climate_types = {}
geni_warm = ["Quercus","Zea"]
geni_cold = ["Abies","Pinus","Picea"]
geni_other = ["Acer"]

for genus in geni_warm:
    climate_types[genus]=0
    
for genus in geni_cold:
    climate_types[genus]=1
    
for genus in geni_other:
    climate_types[genus]=2

In [ ]:
###select which df or data set to operate on
data = simpleCL

#this graphs subplots for each listed species and formats the image

ncols = len(geni)

fig, axs = plt.subplots(1, ncols, sharey= True, sharex = True, figsize = (25,16))

for i,ax in enumerate(axs.ravel()):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    if i == 0:
        ax.invert_yaxis()
    if i > 0:
        ax.get_yaxis().set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)

    genus = geni[i]
    
    if genus not in climate_types.keys():
        col='green' 
    elif climate_types[genus]==0:
        col='red'
    elif climate_types[genus]==1:
        col='blue'
    
    ax.scatter(data[genus]/data["Total Pollen and Spores"],data["Depth"], color = col)
    ax.set_title(genus, fontsize=16, rotation = 0)
    
    space_x = plticker.MultipleLocator(base=.1) # this locator puts ticks at regular intervals
    ax.xaxis.set_major_locator(space_x)
    
    space_y = plticker.MultipleLocator(base=1.5) # this locator puts ticks at regular intervals
    ax.yaxis.set_major_locator(space_y)
    
    ax.set_ylabel("Depth (m)")
    ax.set_xlabel("Percent")
    
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(11) 
    
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(11) 
        
    plt.gca().invert_yaxis()

In [ ]:
###A_counts & RCOUNTS INCLUDING TCT###

geni_compare = ['Pinus','Picea','Quercus','TCT']
ncols = len(geni_compare)

fig, axs = plt.subplots(1, ncols, sharey= True, sharex = True, figsize = (20,8))

for i,ax in enumerate(axs.ravel()):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    if i == 0:
        ax.invert_yaxis()
    if i > 0:
        ax.get_yaxis().set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)

    genus = geni_compare[i]
    ##prevent errors from arising with A_counts data set
    rcounts_clean[genus] = rcounts_clean[genus].apply(float)
    A_counts_clean[genus] = A_counts_clean[genus].apply(float)
    
    ax.scatter(A_counts_clean[genus]/A_counts_clean["Total"],A_counts_clean["Depth"], color = 'black')
    #ax.scatter(A_counts_clean[genus],A_counts_clean["Depth"], color = 'black')
    ax.scatter(rcounts_clean[genus]/rcounts_clean["Total"],rcounts_clean["Depth"], color = 'cyan')
    #ax.scatter(rcounts_clean[genus],rcounts_clean["Depth"], color = 'purple')
    
    plt.gca().invert_yaxis()

    ax.set_title(genus, fontsize=16, rotation = 45)
    
    ax.set_ylabel("Depth (m)")
    ax.set_xlabel("% of TCT Total")
    
    space_x = plticker.MultipleLocator(base=.1) # this locator puts ticks at regular intervals
    ax.xaxis.set_major_locator(space_x)
    
    space_y = plticker.MultipleLocator(base=1) # this locator puts ticks at regular intervals
    ax.yaxis.set_major_locator(space_y)
    
fig.savefig('plots_with_TCT.png')

In [ ]:
####REMOVE TCT FROM DATA AND POLLEN SUM####
A_counts_clean['noTCT_total'] = A_counts_clean['Total'] - A_counts_clean['TCT']
rcounts_clean['noTCT_total'] = rcounts_clean['Total'] - rcounts_clean['TCT']

###A_counts & RCOUNTS WITHOUT TCT###

geni_compare = ['Pinus','Abies','Picea','Tsuga heterophylla','Artemisia','Poaceae','Quercus','Compositae HS',
                'Fraxinus','Alnus','Cyperaceae','Pteridium','Isoetes','Nuphar','Nuphar hydropotes','Typha/Sparganium']
#for shorter lists with only 1 row of subplots: ncols = len(geni_compare))
ncols = round(.5 * len(geni_compare))

fig, axs = plt.subplots(2, ncols, sharey= True, sharex = True, figsize = (50,24))

for i,ax in enumerate(axs.ravel()):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    if i == 0:
        ax.invert_yaxis()
    if i > 0:
        ax.get_yaxis().set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['right'].set_visible(False)

    genus = geni_compare[i]
    ##prevent errors from arising with A_counts data set
    rcounts_clean[genus] = rcounts_clean[genus].apply(float)
    A_counts_clean[genus] = A_counts_clean[genus].apply(float)
    
    ax.scatter(A_counts_clean[genus]/A_counts_clean["noTCT_total"],A_counts_clean["Depth"], color = 'black')
    #ax.scatter(A_counts_clean[genus],A_counts_clean["Depth"], color = 'black')
    ax.scatter(rcounts_clean[genus]/rcounts_clean["noTCT_total"],rcounts_clean["Depth"], color = 'cyan')
    #ax.scatter(rcounts_clean[genus],rcounts_clean["Depth"], color = 'purple')
    
    ax.set_title(genus, fontsize=15, rotation = 30)
    
    ax.set_ylabel("Depth (m)")
    ax.set_xlabel("% of No TCT Total")
    
    space_x = plticker.MultipleLocator(base=.1) # this locator puts ticks at regular intervals
    ax.xaxis.set_major_locator(space_x)
    
    space_y = plticker.MultipleLocator(base=1) # this locator puts ticks at regular intervals
    ax.yaxis.set_major_locator(space_y)

fig.savefig('plots_no_TCT.png')

In [ ]:
##All CL plots (zoomed) without TCT in total

geni_individual = ['Pinus','Abies','Picea','Tsuga heterophylla','Artemisia','Poaceae','Quercus','Compositae HS',
                'Fraxinus','Alnus','Cyperaceae','Pteridium','Isoetes','Nuphar','Nuphar hydropotes','Typha/Sparganium']

for spore in geni_individual:
    fig = plt.figure(figsize = (5,10))
    plt.scatter(A_counts_clean[spore]/A_counts_clean['noTCT_total'],A_counts_clean['Depth'], 
            color = 'black', alpha = 0.8, label = 'A')
    plt.scatter(rcounts_clean[spore]/rcounts_clean['noTCT_total'],rcounts_clean['Depth'], 
            color = 'cyan', alpha = 0.8, label = 'Roger')

    plt.title('CL ' + spore + ' % of Total by Depth NO TCT')
    plt.legend()

    plt.xlabel('% of no TCT total')
    plt.ylabel('Depth (m)')
    
    if spore == 'Pinus' or spore == 'Quercus':
        plt.xticks(np.arange(0, 1, step=.1))
    elif spore == 'Nuphar hydropotes':
        plt.xticks(np.arange(0, 0.3, step=.01))
    elif spore == 'Abies' or spore == 'Tsuga heterophylla' or spore == 'Poaceae' or spore == 'Compositae HS' or spore == 'Cyperaceae' or spore == 'Pteridium' or spore == 'Nuphar' or spore == 'Typha/Sparganium':
        plt.xticks(np.arange(0, 0.1, step=.01))
    else:
        plt.xticks(np.arange(0, 0.25, step=.05))
    
    plt.yticks(np.arange(9, 37, step=1))
    
    plt.gca().invert_yaxis()

    #plt.savefig(spore + '_no_TCT.png')
    plt.show()

In [ ]:
##Pinus with TCT included in the total
fig = plt.figure(figsize = (5,10))

plt.scatter(A_counts_clean['Pinus']/A_counts_clean['Total'],A_counts_clean['Depth'], 
            color = 'black', alpha = 0.8, label = 'A')
plt.scatter(rcounts_clean['Pinus']/rcounts_clean['Total'],rcounts_clean['Depth'], 
            color = 'cyan', alpha = 0.8, label = 'Roger')

plt.title('CL Pinus % of Total by Depth w/TCT')
plt.legend()

plt.xlabel('% of Total')
plt.ylabel('Depth (m)')
plt.yticks(np.arange(9, 37, step=1))
plt.xticks(np.arange(0, 1, step=.1))
plt.gca().invert_yaxis()

plt.savefig('Pinus_with_TCT.png')
plt.show()

In [ ]:
##Pinus without TCT in total
fig = plt.figure(figsize = (5,10))

plt.scatter(A_counts_clean['Pinus']/A_counts_clean['noTCT_total'],A_counts_clean['Depth'], 
            color = 'black', alpha = 0.8, label = 'A')
plt.scatter(rcounts_clean['Pinus']/rcounts_clean['noTCT_total'],rcounts_clean['Depth'], 
            color = 'cyan', alpha = 0.8, label = 'Roger')

plt.title('CL Pinus % of Total by Depth NO TCT')
plt.legend()

plt.xlabel('% of no TCT total')
plt.ylabel('Depth (m)')
plt.yticks(np.arange(9, 37, step=1))
plt.xticks(np.arange(0, 1, step=.1))

plt.gca().invert_yaxis()

plt.savefig('Pinus_no_TCT.png')
plt.show()

In [ ]:
###ADD Indeterminate to Pinus and see if this reconciles counts###
A_counts_clean['Indeterminate Conifer'] = A_counts_clean['Indeterminate Conifer'].apply(float)
A_counts_clean['Pinus + Indeterminate'] = A_counts_clean['Pinus'] + A_counts_clean['Indeterminate Conifer']

##Pinus + indeterminate, without TCT in total
fig = plt.figure(figsize = (5,10))

plt.scatter(A_counts_clean['Pinus + Indeterminate']/A_counts_clean['noTCT_total'],A_counts_clean['Depth'], 
            color = 'black', alpha = 0.8, label = 'A')
plt.scatter(rcounts_clean['Pinus']/rcounts_clean['noTCT_total'],rcounts_clean['Depth'], 
            color = 'cyan', alpha = 0.8, label = 'Roger')

plt.title('CL Pinus % of Total by Depth NO TCT (m indet + m pine)')
plt.legend()

plt.xlabel('% of no TCT total')
plt.ylabel('Depth (m)')
plt.yticks(np.arange(9, 37, step=1))
plt.xticks(np.arange(0, 1, step=.1))

plt.gca().invert_yaxis()

plt.savefig('mPinus-plus-mIndeterminate_V_rPinus_no_TCT.png')
plt.show()

In [ ]:
###STAT test###
#corr_arr = sp.signal.correlate(A_counts_clean['Pinus'],rcounts_clean['Pinus'])
Adepths = A_counts_clean['Depth'].sort_values()
rdepths = rcounts_clean['Depth'].sort_values()
depth_compare = pd.DataFrame(data = rdepths)
depth_compare['A_depth'] = Adepths
depth_compare = depth_compare.rename(columns = {'Depth' : 'r_depth'})
depth_compare

In [ ]:
depth_compare['A_depth_shift'] = depth_compare['A_depth'].shift(periods=-1)
depth_compare

#line up most similar depths
#calculate difference in depths between most similar levels, 

##plum/Bacon diagram for overall chronology
#-compare charcoal to pollen dates

##Add A's intdererminate conifer and m's pine together to see if this matched R's pine more closely
